<a href="https://colab.research.google.com/github/amjad5azx/English-Urdu-Translation/blob/main/language_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Checking if GPU is running or not

!nvidia-smi

Thu Jan  4 13:44:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

text_file_path = "/content/drive/MyDrive/urd.txt"
text_dataset = pd.read_csv(text_file_path, delimiter='\t', header=None, names=['English', 'Urdu', 'License'])
text_dataset

text_dataset = text_dataset[['English', 'Urdu']]
text_dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,English,Urdu
0,Hi.,سلام۔
1,Help!,مدد۔
2,Thanks.,شکریہ۔
3,We won.,ہم جیت گئے۔
4,Beat it.,بھاگ جائو۔
...,...,...
1138,Children who are sixteen years old or younger ...,سولہ سال سے کم عمر کے بچے ٹھیٹر میں داخل نہیں ...
1139,She borrowed the book from him many years ago ...,اس نے اس سے یہ کتاب کئی سال پہلے ادھار لی تھی ...
1140,She asked him to not quit his job because they...,اس نے اسے اپنی نوکری چھوڑنے سے منع کیا کیونکہ ...
1141,Tom would've liked to attend Mary's party. Unf...,ٹام میری کی پارٹی میں شامل تو ہونا چاہتا تھا م...


In [3]:
from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(text_dataset, test_size=0.2, random_state=42)
validation_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

train_dataset = Dataset.from_pandas(train_data)
validation_dataset = Dataset.from_pandas(validation_data)
test_dataset = Dataset.from_pandas(test_data)

custom_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

print(custom_dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['English', 'Urdu', '__index_level_0__'],
        num_rows: 914
    })
    validation: Dataset({
        features: ['English', 'Urdu', '__index_level_0__'],
        num_rows: 114
    })
    test: Dataset({
        features: ['English', 'Urdu', '__index_level_0__'],
        num_rows: 115
    })
})


In [4]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [5]:
model_name = "Helsinki-NLP/opus-mt-en-ur"

In [6]:
# raw_datasets = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")
raw_datasets=custom_dataset_dict

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['English', 'Urdu', '__index_level_0__'],
        num_rows: 914
    })
    validation: Dataset({
        features: ['English', 'Urdu', '__index_level_0__'],
        num_rows: 114
    })
    test: Dataset({
        features: ['English', 'Urdu', '__index_level_0__'],
        num_rows: 115
    })
})

In [8]:
raw_datasets['train'][1]

{'English': "I won't bore you with the details.",
 'Urdu': 'میں تمہیں دیادہ تفصیل بتا کے بور نہیں کرو گا۔',
 '__index_level_0__': 758}

#Preprocessing the data

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [20475, 2, 140, 22, 25, 7166, 67, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[20475, 2, 140, 22, 25, 7166, 67, 0], [416, 22, 474, 7166, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [12]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["میں تمہیں دیادہ تفصیل بتا کے بور نہیں کرو گا۔"]))

{'input_ids': [[13, 145, 133, 2132, 4081, 2256, 6, 23947, 44, 147, 762, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [13]:
max_input_length = 128
max_target_length = 128

source_lang = "English"
target_lang = "Urdu"


def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [14]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[97, 462, 3137, 5, 0], [97, 8041, 128, 678, 5317, 17, 65, 3, 7789, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[13, 2694, 1540, 0], [13, 145, 133, 2132, 4081, 2256, 6, 23947, 44, 147, 762, 0]]}

In [22]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

In [23]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ur.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [24]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [26]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [27]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [28]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [29]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [30]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [31]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=0)

In [21]:
model.save_pretrained("tf_model/")

# Model Testing

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [29]:
pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 4.7 MB/s eta 0:00:00


In [33]:
input_text  = "They won"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[62024    28  9596   296     0 62024]], shape=(1, 6), dtype=int32)


In [34]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))
    print(type(tokenizer.decode(out[0], skip_special_tokens=True)))

وہ جیت گئے
<class 'str'>


# **User Interface**

In [36]:
pip install gradio transformers

In [35]:
!pip install kaleido
!pip install cohere
!pip install openai
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires 

In [33]:
pip install --upgrade pip setuptools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 52.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.


In [35]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_text(input_text):
    try:
        tokenized = tokenizer([input_text], return_tensors='np')
        out = model.generate(**tokenized, max_length=128)
        with tokenizer.as_target_tokenizer():
            generated_text = tokenizer.decode(out[0], skip_special_tokens=True)
            print(generated_text)
        return generated_text
    except Exception as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"

# Create a Gradio interface
iface = gr.Interface(fn=generate_text, inputs="text", outputs="text")
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://46e54be4c0de22511d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [37]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
